In [ ]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv


from sshtunnel import SSHTunnelForwarder
import pymysql

import matplotlib.pyplot as plt

from datetime import datetime
import holidays

import json

current_date = datetime.now().strftime('%Y-%m-%d')

# kr_holidays = holidays.SouthKorea(years=2025)

# 공휴일 획인
# print(kr_holidays.get("2025-08-15"))  

load_dotenv()

SSH_HOST = os.getenv("SSH_HOST")
SSH_PORT = int(os.getenv("SSH_PORT", 22))
SSH_USER = os.getenv("SSH_USER")
SSH_KEY_PATH = os.getenv("SSH_KEY_PATH")

DB_HOST = os.getenv("DB_HOST")
DB_PORT = int(os.getenv("DB_PORT", 3306))
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_ORDER_SERVICE = os.getenv("DB_ORDER_SERVICE")

def query_db(query, params=None):
    # SSH 키 로드
    ssh_key = SSH_KEY_PATH
    
    with SSHTunnelForwarder(
        (SSH_HOST, SSH_PORT),
        ssh_username=SSH_USER,
        ssh_pkey=ssh_key,  # 로드된 키 객체 또는 파일 경로
        remote_bind_address=(DB_HOST, DB_PORT)
    ) as tunnel:
        with pymysql.connect(
            host='127.0.0.1',
            port=tunnel.local_bind_port,
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_ORDER_SERVICE,
            charset='utf8mb4'
        ) as conn:    
            try:
                df = pd.read_sql(query, conn, params=params)
                if df.empty and len(df.columns) > 0:
                    with conn.cursor() as cursor:
                        cursor.execute(query, params)
                        rows = cursor.fetchall()
                        if rows:
                            df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
            except Exception as e:
                print(f"[ERROR] 데이터 조회 중 오류: {e}")  # f-string 수정
                raise
    return df             